![alt text](./image_files/pageheader_rose2_babies.jpg)

# Data Science in Medicine using Python

### Author: Dr Gusztav Belteki

## 1. Review of homework: slicing and dicing in Python

Subsetting and indexing pandas DataFrames

In [1]:
a = 42; b = 'Hello World'

In [2]:
dir()

['In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit']

In [3]:
import os
import pandas as pd

filenames = ['CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip', ]

data = {}

for file in filenames:
    path = os.path.join('data', file,)
    data[file] = pd.read_csv(path)
    data

In [4]:
data_all = pd.concat(data)

In [5]:
data_all

Time [ms]  \
CsvLogBase_2020-11-02_134238.904_slow_Measureme... 0       1604324559904   
                                                   1       1604324560029   
                                                   2       1604324560951   
                                                   3       1604324561060   
                                                   4       1604324561935   
...                                                                  ...   
                                                   689431  1604669233906   
                                                   689432  1604669234031   
                                                   689433  1604669234906   
                                                   689434  1604669235046   
                                                   689435  1604669235937   

                                                                 Date  \
CsvLogBase_2020-11-02_134238.904_slow_Measureme... 0       2020-11-02   
                                                   1       2020-11-02   
                                                   2       2020-11-02   
                                                   3       2020-11-02   
                                                   4       2020-11-02   
...                                                               ...   
                                                   689431  2020-11-06   
                                                   689432  2020-11-06   
                                                   689433  2020-11-06   
                                                   689434  2020-11-06   
                                                   689435  2020-11-06   

                                                                   Time  \
CsvLogBase_2020-11-02_134238.904_slow_Measureme... 0       13:42:39.904   
                                                   1       13:42:40.029   
                                                   2       13:42:40.951   
                                                   3       13:42:41.060   
                                                   4       13:42:41.935   
...                                                                 ...   
                                                   689431  13:27:13.906   
                                                   689432  13:27:14.031   
                                                   689433  13:27:14.906   
                                                   689434  13:27:15.046   
                                                   689435  13:27:15.937   

                                                           Rel.Time [s]  \
CsvLogBase_2020-11-02_134238.904_slow_Measureme... 0                  0   
                                                   1                  0   
                                                   2                  1   
                                                   3                  1   
                                                   4                  2   
...                                                                 ...   
                                                   689431        344674   
                                                   689432        344674   
                                                   689433        344675   
                                                   689434        344675   
                                                   689435        344676   

                                                           5001|MVe [L/min]  \
CsvLogBase_2020-11-02_134238.904_slow_Measureme... 0                   0.20   
                                                   1                    NaN   
                                                   2                   0.20   
                                                   3                    NaN   
                                                   4                   0.20 

In [6]:
import os
import pandas as pd

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip',)
data = pd.read_csv(path)
data

,Time [ms],Date,Time,Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,1604324559904,2020-11-02,13:42:39.904,0,0.20,0.21,0.21,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1604324560029,2020-11-02,13:42:40.029,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.00,NaN,NaN,NaN
2,1604324560951,2020-11-02,13:42:40.951,1,0.20,0.21,0.22,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1604324561060,2020-11-02,13:42:41.060,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.00,NaN,NaN,NaN
4,1604324561935,2020-11-02,13:42:41.935,2,0.20,0.21,0.26,154.0,0.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689431,1604669233906,2020-11-06,13:27:13.906,344674,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689432,1604669234031,2020-11-06,13:27:14.031,344674,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.11,0.75,3.1,2.6,18.0,0.02,NaN,NaN,NaN
689433,1604669234906,2020-11-06,13:27:14.906,344675,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689434,1604669235046,2020-11-06,13:27:15.046,344675,NaN,NaN,NaN,NaN,NaN,NaN,...,0.01,0.15,0.75,4.1,2.6,19.0,0.01,NaN,NaN,NaN


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689436 entries, 0 to 689435
Data columns (total 47 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Time [ms]                        689436 non-null  int64  
 1   Date                             689436 non-null  object 
 2   Time                             689436 non-null  object 
 3   Rel.Time [s]                     689436 non-null  int64  
 4   5001|MVe [L/min]                 344633 non-null  float64
 5   5001|MVi [L/min]                 344633 non-null  float64
 6   5001|Cdyn [L/bar]                344073 non-null  float64
 7   5001|R [mbar/L/s]                342099 non-null  float64
 8   5001|MVespon [L/min]             344633 non-null  float64
 9   5001|Rpat [mbar/L/s]             341398 non-null  float64
 10  5001|MVemand [L/min]             344633 non-null  float64
 11  5001|FlowDev [L/min]             344677 non-null  float64
 12  50

In [8]:
data.head(5)

,Time [ms],Date,Time,Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,1604324559904,2020-11-02,13:42:39.904,0,0.2,0.21,0.21,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1604324560029,2020-11-02,13:42:40.029,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.0,NaN,NaN,NaN
2,1604324560951,2020-11-02,13:42:40.951,1,0.2,0.21,0.22,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1604324561060,2020-11-02,13:42:41.060,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.0,NaN,NaN,NaN
4,1604324561935,2020-11-02,13:42:41.935,2,0.2,0.21,0.26,154.0,0.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Select the third row only

selection = data
selection.loc[0]

Time [ms]                          1604324559904
Date                                  2020-11-02
Time                                13:42:39.904
Rel.Time [s]                                   0
5001|MVe [L/min]                             0.2
5001|MVi [L/min]                            0.21
5001|Cdyn [L/bar]                           0.21
5001|R [mbar/L/s]                            146
5001|MVespon [L/min]                           0
5001|Rpat [mbar/L/s]                         128
5001|MVemand [L/min]                         0.2
5001|FlowDev [L/min]                         6.9
5001|VTmand [mL]                             3.4
5001|r2 [no unit]                           0.98
5001|VTispon [mL]                              0
5001|Pmin [mbar]                             4.4
5001|Pmean [mbar]                             10
5001|PEEP [mbar]                             4.6
5001|RRmand [1/min]                           61
5001|PIP [mbar]                               22
5001|VTmand [L]     

In [10]:
# Select the "MVe [L/min]" column only"

selection = data['5001|MVe [L/min]']
selection

0         0.20
1          NaN
2         0.20
3          NaN
4         0.20
          ... 
689431    0.22
689432     NaN
689433    0.22
689434     NaN
689435    0.22
Name: 5001|MVe [L/min], Length: 689436, dtype: float64

In [11]:
data.columns

Index(['Time [ms]', 'Date', 'Time', 'Rel.Time [s]', '5001|MVe [L/min]',
       '5001|MVi [L/min]', '5001|Cdyn [L/bar]', '5001|R [mbar/L/s]',
       '5001|MVespon [L/min]', '5001|Rpat [mbar/L/s]', '5001|MVemand [L/min]',
       '5001|FlowDev [L/min]', '5001|VTmand [mL]', '5001|r2 [no unit]',
       '5001|VTispon [mL]', '5001|Pmin [mbar]', '5001|Pmean [mbar]',
       '5001|PEEP [mbar]', '5001|RRmand [1/min]', '5001|PIP [mbar]',
       '5001|VTmand [L]', '5001|VTspon [L]', '5001|VTemand [mL]',
       '5001|VTespon [mL]', '5001|VTimand [mL]', '5001|VT [mL]',
       '5001|% leak [%]', '5001|RRspon [1/min]', '5001|% MVspon [%]',
       '5001|MV [L/min]', '5001|RRtrig [1/min]', '5001|RR [1/min]',
       '5001|I (I:E) [no unit]', '5001|E (I:E) [no unit]', '5001|FiO2 [%]',
       '5001|VTspon [mL]', '5001|E [mbar/L]', '5001|TC [s]', '5001|TCe [s]',
       '5001|C20/Cdyn [no unit]', '5001|VTe [mL]', '5001|VTi [mL]',
       '5001|EIP [mbar]', '5001|MVleak [L/min]', '5001|Tispon [s]',
       '5001

In [12]:
# Select the "MVe [L/min]" and "MVi [L/min]" columns only

columns_to_keep = ['5001|MVe [L/min]', '5001|MVi [L/min]']
selection = data[columns_to_keep]
selection

,5001|MVe [L/min],5001|MVi [L/min]
0,0.20,0.21
1,NaN,NaN
2,0.20,0.21
3,NaN,NaN
4,0.20,0.21
...,...,...
689431,0.22,0.22
689432,NaN,NaN
689433,0.22,0.22
689434,NaN,NaN


In [13]:
# Select the 'MVe [L/min]' value from the third row

selection = data.iloc[2]['5001|MVe [L/min]']
selection

0.2

In [14]:
type(selection)

numpy.float64

##  End-to-end analysis of tabular (two-dimensional data) using pandas

In [15]:
pd.read_csv?

You can also look it up on the internetm

In [16]:
import os
import pandas as pd

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip',)
data = pd.read_csv(path)
data

,Time [ms],Date,Time,Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,1604324559904,2020-11-02,13:42:39.904,0,0.20,0.21,0.21,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1604324560029,2020-11-02,13:42:40.029,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.00,NaN,NaN,NaN
2,1604324560951,2020-11-02,13:42:40.951,1,0.20,0.21,0.22,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1604324561060,2020-11-02,13:42:41.060,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.00,NaN,NaN,NaN
4,1604324561935,2020-11-02,13:42:41.935,2,0.20,0.21,0.26,154.0,0.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689431,1604669233906,2020-11-06,13:27:13.906,344674,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689432,1604669234031,2020-11-06,13:27:14.031,344674,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.11,0.75,3.1,2.6,18.0,0.02,NaN,NaN,NaN
689433,1604669234906,2020-11-06,13:27:14.906,344675,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689434,1604669235046,2020-11-06,13:27:15.046,344675,NaN,NaN,NaN,NaN,NaN,NaN,...,0.01,0.15,0.75,4.1,2.6,19.0,0.01,NaN,NaN,NaN


In [17]:
len(data)

689436

In [18]:
data.shape

(689436, 47)

In [19]:
data.ndim

2

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689436 entries, 0 to 689435
Data columns (total 47 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Time [ms]                        689436 non-null  int64  
 1   Date                             689436 non-null  object 
 2   Time                             689436 non-null  object 
 3   Rel.Time [s]                     689436 non-null  int64  
 4   5001|MVe [L/min]                 344633 non-null  float64
 5   5001|MVi [L/min]                 344633 non-null  float64
 6   5001|Cdyn [L/bar]                344073 non-null  float64
 7   5001|R [mbar/L/s]                342099 non-null  float64
 8   5001|MVespon [L/min]             344633 non-null  float64
 9   5001|Rpat [mbar/L/s]             341398 non-null  float64
 10  5001|MVemand [L/min]             344633 non-null  float64
 11  5001|FlowDev [L/min]             344677 non-null  float64
 12  50

In [21]:
round(data.describe(), 2)

,Time [ms],Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],5001|MVemand [L/min],5001|FlowDev [L/min],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
count,6.894360e+05,689436.00,344633.00,344633.00,344073.00,342099.00,344633.00,341398.00,344633.00,344677.00,...,344051.00,344566.00,339993.00,344566.00,344570.00,344676.00,344632.00,10015.00,9071.00,9071.00
mean,1.604497e+12,172337.58,0.22,0.26,0.25,274.04,0.00,257.38,0.22,7.13,...,0.07,0.14,0.72,3.72,4.34,23.86,0.05,0.05,1.02,7.19
std,9.949949e+07,99499.48,0.03,0.14,0.17,165.36,0.00,167.91,0.03,0.72,...,0.11,0.05,0.36,0.78,1.05,4.35,0.32,0.01,0.22,1.89
min,1.604325e+12,0.00,0.00,0.00,0.00,29.40,0.00,15.80,0.00,6.40,...,0.00,0.01,0.23,0.60,0.60,1.80,0.00,0.04,1.00,1.00
25%,1.604411e+12,86168.00,0.21,0.23,0.21,162.00,0.00,143.00,0.21,6.90,...,0.04,0.11,0.57,3.30,3.80,21.00,0.01,0.04,1.00,7.40
50%,1.604497e+12,172337.00,0.23,0.26,0.24,213.00,0.00,195.00,0.23,7.00,...,0.05,0.12,0.64,3.80,4.30,22.00,0.02,0.05,1.00,8.00
75%,1.604583e+12,258506.25,0.24,0.28,0.27,328.00,0.00,311.00,0.24,7.20,...,0.08,0.16,0.74,4.00,4.80,25.00,0.04,0.05,1.00,8.10
max,1.604669e+12,344676.00,0.49,18.60,57.40,1000.00,0.05,1000.00,0.49,31.20,...,28.90,0.77,4.97,13.80,143.00,53.00,27.60,0.09,10.40,9.50


In [22]:
data.head()

,Time [ms],Date,Time,Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,1604324559904,2020-11-02,13:42:39.904,0,0.2,0.21,0.21,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1604324560029,2020-11-02,13:42:40.029,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.0,NaN,NaN,NaN
2,1604324560951,2020-11-02,13:42:40.951,1,0.2,0.21,0.22,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1604324561060,2020-11-02,13:42:41.060,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.0,NaN,NaN,NaN
4,1604324561935,2020-11-02,13:42:41.935,2,0.2,0.21,0.26,154.0,0.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
data.tail()

,Time [ms],Date,Time,Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
689431,1604669233906,2020-11-06,13:27:13.906,344674,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689432,1604669234031,2020-11-06,13:27:14.031,344674,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.11,0.75,3.1,2.6,18.0,0.02,NaN,NaN,NaN
689433,1604669234906,2020-11-06,13:27:14.906,344675,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689434,1604669235046,2020-11-06,13:27:15.046,344675,NaN,NaN,NaN,NaN,NaN,NaN,...,0.01,0.15,0.75,4.1,2.6,19.0,0.01,NaN,NaN,NaN
689435,1604669235937,2020-11-06,13:27:15.937,344676,0.22,0.22,0.05,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### What is the problem with these data

- Indexed by numbers only (uninformative, it should be indexed by date and time)
- Date and time are in separate columnns
- Date and time formats are not appropriate
- Column names are too long and difficult to read
- Lots of `na` values 
- half of every row is empty
- Some columns have barely any informative values
- Some values are not meaningful (e.g. tidal volume should be mL/kg not mL)

*We will deal with all these issues*

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689436 entries, 0 to 689435
Data columns (total 47 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Time [ms]                        689436 non-null  int64  
 1   Date                             689436 non-null  object 
 2   Time                             689436 non-null  object 
 3   Rel.Time [s]                     689436 non-null  int64  
 4   5001|MVe [L/min]                 344633 non-null  float64
 5   5001|MVi [L/min]                 344633 non-null  float64
 6   5001|Cdyn [L/bar]                344073 non-null  float64
 7   5001|R [mbar/L/s]                342099 non-null  float64
 8   5001|MVespon [L/min]             344633 non-null  float64
 9   5001|Rpat [mbar/L/s]             341398 non-null  float64
 10  5001|MVemand [L/min]             344633 non-null  float64
 11  5001|FlowDev [L/min]             344677 non-null  float64
 12  50

### 1. Convert the `date` and `time` columns to appropriate format

Let us time how long the import takes

In [25]:
# Let us time how long the import takes

In [26]:
%%time 

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip')
data = pd.read_csv(path)
data

CPU times: user 2.48 s, sys: 306 ms, total: 2.79 s
Wall time: 2.79 s


,Time [ms],Date,Time,Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,1604324559904,2020-11-02,13:42:39.904,0,0.20,0.21,0.21,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1604324560029,2020-11-02,13:42:40.029,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.00,NaN,NaN,NaN
2,1604324560951,2020-11-02,13:42:40.951,1,0.20,0.21,0.22,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1604324561060,2020-11-02,13:42:41.060,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.00,NaN,NaN,NaN
4,1604324561935,2020-11-02,13:42:41.935,2,0.20,0.21,0.26,154.0,0.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689431,1604669233906,2020-11-06,13:27:13.906,344674,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689432,1604669234031,2020-11-06,13:27:14.031,344674,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.11,0.75,3.1,2.6,18.0,0.02,NaN,NaN,NaN
689433,1604669234906,2020-11-06,13:27:14.906,344675,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689434,1604669235046,2020-11-06,13:27:15.046,344675,NaN,NaN,NaN,NaN,NaN,NaN,...,0.01,0.15,0.75,4.1,2.6,19.0,0.01,NaN,NaN,NaN


In [27]:
pd.read_csv?

This takes much longer

In [28]:
%%time 

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip')
lst = ['Date', 'Time']
data = pd.read_csv(path, parse_dates = lst)
data

CPU times: user 30.1 s, sys: 373 ms, total: 30.5 s
Wall time: 30.6 s


,Time [ms],Date,Time,Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,1604324559904,2020-11-02,2021-04-16 13:42:39.904,0,0.20,0.21,0.21,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1604324560029,2020-11-02,2021-04-16 13:42:40.029,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.00,NaN,NaN,NaN
2,1604324560951,2020-11-02,2021-04-16 13:42:40.951,1,0.20,0.21,0.22,146.0,0.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1604324561060,2020-11-02,2021-04-16 13:42:41.060,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.00,NaN,NaN,NaN
4,1604324561935,2020-11-02,2021-04-16 13:42:41.935,2,0.20,0.21,0.26,154.0,0.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689431,1604669233906,2020-11-06,2021-04-16 13:27:13.906,344674,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689432,1604669234031,2020-11-06,2021-04-16 13:27:14.031,344674,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.11,0.75,3.1,2.6,18.0,0.02,NaN,NaN,NaN
689433,1604669234906,2020-11-06,2021-04-16 13:27:14.906,344675,0.22,0.22,0.21,164.0,0.0,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689434,1604669235046,2020-11-06,2021-04-16 13:27:15.046,344675,NaN,NaN,NaN,NaN,NaN,NaN,...,0.01,0.15,0.75,4.1,2.6,19.0,0.01,NaN,NaN,NaN


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689436 entries, 0 to 689435
Data columns (total 47 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Time [ms]                        689436 non-null  int64         
 1   Date                             689436 non-null  datetime64[ns]
 2   Time                             689436 non-null  datetime64[ns]
 3   Rel.Time [s]                     689436 non-null  int64         
 4   5001|MVe [L/min]                 344633 non-null  float64       
 5   5001|MVi [L/min]                 344633 non-null  float64       
 6   5001|Cdyn [L/bar]                344073 non-null  float64       
 7   5001|R [mbar/L/s]                342099 non-null  float64       
 8   5001|MVespon [L/min]             344633 non-null  float64       
 9   5001|Rpat [mbar/L/s]             341398 non-null  float64       
 10  5001|MVemand [L/min]             344633 non-

There must be a better way !!!

Google: **"How to combine date and time columns in pandas"**

In [30]:
%%time

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip')
data = pd.read_csv(path, parse_dates = [['Date', 'Time']])
data

CPU times: user 2.92 s, sys: 387 ms, total: 3.31 s
Wall time: 3.37 s


,Date_Time,Time [ms],Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],5001|MVemand [L/min],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,2020-11-02 13:42:39.904,1604324559904,0,0.20,0.21,0.21,146.0,0.0,128.0,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-02 13:42:40.029,1604324560029,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.00,NaN,NaN,NaN
2,2020-11-02 13:42:40.951,1604324560951,1,0.20,0.21,0.22,146.0,0.0,128.0,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-02 13:42:41.060,1604324561060,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.00,NaN,NaN,NaN
4,2020-11-02 13:42:41.935,1604324561935,2,0.20,0.21,0.26,154.0,0.0,136.0,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689431,2020-11-06 13:27:13.906,1604669233906,344674,0.22,0.22,0.21,164.0,0.0,150.0,0.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689432,2020-11-06 13:27:14.031,1604669234031,344674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.11,0.75,3.1,2.6,18.0,0.02,NaN,NaN,NaN
689433,2020-11-06 13:27:14.906,1604669234906,344675,0.22,0.22,0.21,164.0,0.0,150.0,0.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689434,2020-11-06 13:27:15.046,1604669235046,344675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.01,0.15,0.75,4.1,2.6,19.0,0.01,NaN,NaN,NaN


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689436 entries, 0 to 689435
Data columns (total 46 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Date_Time                        689436 non-null  datetime64[ns]
 1   Time [ms]                        689436 non-null  int64         
 2   Rel.Time [s]                     689436 non-null  int64         
 3   5001|MVe [L/min]                 344633 non-null  float64       
 4   5001|MVi [L/min]                 344633 non-null  float64       
 5   5001|Cdyn [L/bar]                344073 non-null  float64       
 6   5001|R [mbar/L/s]                342099 non-null  float64       
 7   5001|MVespon [L/min]             344633 non-null  float64       
 8   5001|Rpat [mbar/L/s]             341398 non-null  float64       
 9   5001|MVemand [L/min]             344633 non-null  float64       
 10  5001|FlowDev [L/min]             344677 non-

### 2. Set the `Date_Time` column as index

In [32]:
data.head()

,Date_Time,Time [ms],Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],5001|MVemand [L/min],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
0,2020-11-02 13:42:39.904,1604324559904,0,0.2,0.21,0.21,146.0,0.0,128.0,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-02 13:42:40.029,1604324560029,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.0,NaN,NaN,NaN
2,2020-11-02 13:42:40.951,1604324560951,1,0.2,0.21,0.22,146.0,0.0,128.0,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-02 13:42:41.060,1604324561060,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.0,NaN,NaN,NaN
4,2020-11-02 13:42:41.935,1604324561935,2,0.2,0.21,0.26,154.0,0.0,136.0,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
data.index

RangeIndex(start=0, stop=689436, step=1)

In [34]:
list(data.index)[:15]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [35]:
data = data.set_index('Date_Time') # Do not use inplace modifications
data

,Time [ms],Rel.Time [s],5001|MVe [L/min],5001|MVi [L/min],5001|Cdyn [L/bar],5001|R [mbar/L/s],5001|MVespon [L/min],5001|Rpat [mbar/L/s],5001|MVemand [L/min],5001|FlowDev [L/min],...,5001|TC [s],5001|TCe [s],5001|C20/Cdyn [no unit],5001|VTe [mL],5001|VTi [mL],5001|EIP [mbar],5001|MVleak [L/min],5001|Tispon [s],5001|I:Espon (I-Part) [no unit],5001|I:Espon (E-Part) [no unit]
Date_Time,,,,,,,,,,,,,,,,,,,,,
2020-11-02 13:42:39.904,1604324559904,0,0.20,0.21,0.21,146.0,0.0,128.0,0.20,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:40.029,1604324560029,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.00,NaN,NaN,NaN
2020-11-02 13:42:40.951,1604324560951,1,0.20,0.21,0.22,146.0,0.0,128.0,0.20,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:41.060,1604324561060,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.00,NaN,NaN,NaN
2020-11-02 13:42:41.935,1604324561935,2,0.20,0.21,0.26,154.0,0.0,136.0,0.20,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-06 13:27:13.906,1604669233906,344674,0.22,0.22,0.21,164.0,0.0,150.0,0.22,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-06 13:27:14.031,1604669234031,344674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.11,0.75,3.1,2.6,18.0,0.02,NaN,NaN,NaN
2020-11-06 13:27:14.906,1604669234906,344675,0.22,0.22,0.21,164.0,0.0,150.0,0.22,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
data.index

DatetimeIndex(['2020-11-02 13:42:39.904000', '2020-11-02 13:42:40.029000',
               '2020-11-02 13:42:40.951000', '2020-11-02 13:42:41.060000',
               '2020-11-02 13:42:41.935000', '2020-11-02 13:42:42.060000',
               '2020-11-02 13:42:42.888000', '2020-11-02 13:42:43.028000',
               '2020-11-02 13:42:43.919000', '2020-11-02 13:42:44.044000',
               ...
               '2020-11-06 13:27:11.062000', '2020-11-06 13:27:11.921000',
               '2020-11-06 13:27:12.046000', '2020-11-06 13:27:12.937000',
               '2020-11-06 13:27:13.062000', '2020-11-06 13:27:13.906000',
               '2020-11-06 13:27:14.031000', '2020-11-06 13:27:14.906000',
               '2020-11-06 13:27:15.046000', '2020-11-06 13:27:15.937000'],
              dtype='datetime64[ns]', name='Date_Time', length=689436, freq=None)

### 3. Change the clumsy column names

In [37]:
data_columns = ['Time [ms]', 'Rel.Time [s]', 'MVe [L/min]', 'MVi [L/min]', 'Cdyn [L/bar]', 'R [mbar/L/s]', 'MVespon [L/min]', 'Rpat [mbar/L/s]', 'MVemand [L/min]', 'FlowDev [L/min]', 'VTmand [mL]', 'r2 [no unit]', 'VTispon [mL]', 'Pmin [mbar]', 'Pmean [mbar]', 'PEEP [mbar]', 'RRmand [1/min]', 'PIP [mbar]', 'VTmand [L]', 'VTspon [L]', 'VTemand [mL]', 'VTespon [mL]', 'VTimand [mL]', 'VT [mL]', '% leak [%]', 'RRspon [1/min]', '% MVspon [%]', 'MV [L/min]', 'RRtrig [1/min]', 'RR [1/min]', 'I (I:E) [no unit]', 'E (I:E) [no unit]', 'FiO2 [%]', 'VTspon [mL]', 'E [mbar/L]', 'TC [s]', 'TCe [s]', 'C20/Cdyn [no unit]', 'VTe [mL]', 'VTi [mL]', 'EIP [mbar]', 'MVleak [L/min]', 'Tispon [s]', 'I:Espon (I-Part) [no unit]', 'I:Espon (E-Part) [no unit]']

In [38]:
data.columns

Index(['Time [ms]', 'Rel.Time [s]', '5001|MVe [L/min]', '5001|MVi [L/min]',
       '5001|Cdyn [L/bar]', '5001|R [mbar/L/s]', '5001|MVespon [L/min]',
       '5001|Rpat [mbar/L/s]', '5001|MVemand [L/min]', '5001|FlowDev [L/min]',
       '5001|VTmand [mL]', '5001|r2 [no unit]', '5001|VTispon [mL]',
       '5001|Pmin [mbar]', '5001|Pmean [mbar]', '5001|PEEP [mbar]',
       '5001|RRmand [1/min]', '5001|PIP [mbar]', '5001|VTmand [L]',
       '5001|VTspon [L]', '5001|VTemand [mL]', '5001|VTespon [mL]',
       '5001|VTimand [mL]', '5001|VT [mL]', '5001|% leak [%]',
       '5001|RRspon [1/min]', '5001|% MVspon [%]', '5001|MV [L/min]',
       '5001|RRtrig [1/min]', '5001|RR [1/min]', '5001|I (I:E) [no unit]',
       '5001|E (I:E) [no unit]', '5001|FiO2 [%]', '5001|VTspon [mL]',
       '5001|E [mbar/L]', '5001|TC [s]', '5001|TCe [s]',
       '5001|C20/Cdyn [no unit]', '5001|VTe [mL]', '5001|VTi [mL]',
       '5001|EIP [mbar]', '5001|MVleak [L/min]', '5001|Tispon [s]',
       '5001|I:Espon (I-Part

We could just replace it with 

`data.columns = ['...', '...', '...']` 

but that is error prone

In [39]:
# Welcome to list comprehensions

new_columns_1 = [item for item in data.columns]
print(new_columns_1)

['Time [ms]', 'Rel.Time [s]', '5001|MVe [L/min]', '5001|MVi [L/min]', '5001|Cdyn [L/bar]', '5001|R [mbar/L/s]', '5001|MVespon [L/min]', '5001|Rpat [mbar/L/s]', '5001|MVemand [L/min]', '5001|FlowDev [L/min]', '5001|VTmand [mL]', '5001|r2 [no unit]', '5001|VTispon [mL]', '5001|Pmin [mbar]', '5001|Pmean [mbar]', '5001|PEEP [mbar]', '5001|RRmand [1/min]', '5001|PIP [mbar]', '5001|VTmand [L]', '5001|VTspon [L]', '5001|VTemand [mL]', '5001|VTespon [mL]', '5001|VTimand [mL]', '5001|VT [mL]', '5001|% leak [%]', '5001|RRspon [1/min]', '5001|% MVspon [%]', '5001|MV [L/min]', '5001|RRtrig [1/min]', '5001|RR [1/min]', '5001|I (I:E) [no unit]', '5001|E (I:E) [no unit]', '5001|FiO2 [%]', '5001|VTspon [mL]', '5001|E [mbar/L]', '5001|TC [s]', '5001|TCe [s]', '5001|C20/Cdyn [no unit]', '5001|VTe [mL]', '5001|VTi [mL]', '5001|EIP [mbar]', '5001|MVleak [L/min]', '5001|Tispon [s]', '5001|I:Espon (I-Part) [no unit]', '5001|I:Espon (E-Part) [no unit]']


In [40]:
new_columns_2 = [item[5:] for item in data.columns]
print(new_columns_2)

['[ms]', 'ime [s]', 'MVe [L/min]', 'MVi [L/min]', 'Cdyn [L/bar]', 'R [mbar/L/s]', 'MVespon [L/min]', 'Rpat [mbar/L/s]', 'MVemand [L/min]', 'FlowDev [L/min]', 'VTmand [mL]', 'r2 [no unit]', 'VTispon [mL]', 'Pmin [mbar]', 'Pmean [mbar]', 'PEEP [mbar]', 'RRmand [1/min]', 'PIP [mbar]', 'VTmand [L]', 'VTspon [L]', 'VTemand [mL]', 'VTespon [mL]', 'VTimand [mL]', 'VT [mL]', '% leak [%]', 'RRspon [1/min]', '% MVspon [%]', 'MV [L/min]', 'RRtrig [1/min]', 'RR [1/min]', 'I (I:E) [no unit]', 'E (I:E) [no unit]', 'FiO2 [%]', 'VTspon [mL]', 'E [mbar/L]', 'TC [s]', 'TCe [s]', 'C20/Cdyn [no unit]', 'VTe [mL]', 'VTi [mL]', 'EIP [mbar]', 'MVleak [L/min]', 'Tispon [s]', 'I:Espon (I-Part) [no unit]', 'I:Espon (E-Part) [no unit]']


In [41]:
new_columns_3 = [item[5:] for item in data.columns if item.startswith('5001')]
print(new_columns_3)

['MVe [L/min]', 'MVi [L/min]', 'Cdyn [L/bar]', 'R [mbar/L/s]', 'MVespon [L/min]', 'Rpat [mbar/L/s]', 'MVemand [L/min]', 'FlowDev [L/min]', 'VTmand [mL]', 'r2 [no unit]', 'VTispon [mL]', 'Pmin [mbar]', 'Pmean [mbar]', 'PEEP [mbar]', 'RRmand [1/min]', 'PIP [mbar]', 'VTmand [L]', 'VTspon [L]', 'VTemand [mL]', 'VTespon [mL]', 'VTimand [mL]', 'VT [mL]', '% leak [%]', 'RRspon [1/min]', '% MVspon [%]', 'MV [L/min]', 'RRtrig [1/min]', 'RR [1/min]', 'I (I:E) [no unit]', 'E (I:E) [no unit]', 'FiO2 [%]', 'VTspon [mL]', 'E [mbar/L]', 'TC [s]', 'TCe [s]', 'C20/Cdyn [no unit]', 'VTe [mL]', 'VTi [mL]', 'EIP [mbar]', 'MVleak [L/min]', 'Tispon [s]', 'I:Espon (I-Part) [no unit]', 'I:Espon (E-Part) [no unit]']


In [42]:
# The expression to the right of `=` is evaluated firs (before assignment)

new_columns_3 = ['Time [ms]', 'Rel.Time [s]'] + new_columns_3
print(new_columns_3)

['Time [ms]', 'Rel.Time [s]', 'MVe [L/min]', 'MVi [L/min]', 'Cdyn [L/bar]', 'R [mbar/L/s]', 'MVespon [L/min]', 'Rpat [mbar/L/s]', 'MVemand [L/min]', 'FlowDev [L/min]', 'VTmand [mL]', 'r2 [no unit]', 'VTispon [mL]', 'Pmin [mbar]', 'Pmean [mbar]', 'PEEP [mbar]', 'RRmand [1/min]', 'PIP [mbar]', 'VTmand [L]', 'VTspon [L]', 'VTemand [mL]', 'VTespon [mL]', 'VTimand [mL]', 'VT [mL]', '% leak [%]', 'RRspon [1/min]', '% MVspon [%]', 'MV [L/min]', 'RRtrig [1/min]', 'RR [1/min]', 'I (I:E) [no unit]', 'E (I:E) [no unit]', 'FiO2 [%]', 'VTspon [mL]', 'E [mbar/L]', 'TC [s]', 'TCe [s]', 'C20/Cdyn [no unit]', 'VTe [mL]', 'VTi [mL]', 'EIP [mbar]', 'MVleak [L/min]', 'Tispon [s]', 'I:Espon (I-Part) [no unit]', 'I:Espon (E-Part) [no unit]']


In [43]:
data.columns = new_columns_3
data.columns

Index(['Time [ms]', 'Rel.Time [s]', 'MVe [L/min]', 'MVi [L/min]',
       'Cdyn [L/bar]', 'R [mbar/L/s]', 'MVespon [L/min]', 'Rpat [mbar/L/s]',
       'MVemand [L/min]', 'FlowDev [L/min]', 'VTmand [mL]', 'r2 [no unit]',
       'VTispon [mL]', 'Pmin [mbar]', 'Pmean [mbar]', 'PEEP [mbar]',
       'RRmand [1/min]', 'PIP [mbar]', 'VTmand [L]', 'VTspon [L]',
       'VTemand [mL]', 'VTespon [mL]', 'VTimand [mL]', 'VT [mL]', '% leak [%]',
       'RRspon [1/min]', '% MVspon [%]', 'MV [L/min]', 'RRtrig [1/min]',
       'RR [1/min]', 'I (I:E) [no unit]', 'E (I:E) [no unit]', 'FiO2 [%]',
       'VTspon [mL]', 'E [mbar/L]', 'TC [s]', 'TCe [s]', 'C20/Cdyn [no unit]',
       'VTe [mL]', 'VTi [mL]', 'EIP [mbar]', 'MVleak [L/min]', 'Tispon [s]',
       'I:Espon (I-Part) [no unit]', 'I:Espon (E-Part) [no unit]'],
      dtype='object')

In [44]:
data.head()

,Time [ms],Rel.Time [s],MVe [L/min],MVi [L/min],Cdyn [L/bar],R [mbar/L/s],MVespon [L/min],Rpat [mbar/L/s],MVemand [L/min],FlowDev [L/min],...,TC [s],TCe [s],C20/Cdyn [no unit],VTe [mL],VTi [mL],EIP [mbar],MVleak [L/min],Tispon [s],I:Espon (I-Part) [no unit],I:Espon (E-Part) [no unit]
Date_Time,,,,,,,,,,,,,,,,,,,,,
2020-11-02 13:42:39.904,1604324559904,0,0.2,0.21,0.21,146.0,0.0,128.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:40.029,1604324560029,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:40.951,1604324560951,1,0.2,0.21,0.22,146.0,0.0,128.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:41.060,1604324561060,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:41.935,1604324561935,2,0.2,0.21,0.26,154.0,0.0,136.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4. Combine consecutive columns as they contain complementary data

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 689436 entries, 2020-11-02 13:42:39.904000 to 2020-11-06 13:27:15.937000
Data columns (total 45 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Time [ms]                   689436 non-null  int64  
 1   Rel.Time [s]                689436 non-null  int64  
 2   MVe [L/min]                 344633 non-null  float64
 3   MVi [L/min]                 344633 non-null  float64
 4   Cdyn [L/bar]                344073 non-null  float64
 5   R [mbar/L/s]                342099 non-null  float64
 6   MVespon [L/min]             344633 non-null  float64
 7   Rpat [mbar/L/s]             341398 non-null  float64
 8   MVemand [L/min]             344633 non-null  float64
 9   FlowDev [L/min]             344677 non-null  float64
 10  VTmand [mL]                 344567 non-null  float64
 11  r2 [no unit]                344538 non-null  float64
 12  VTispon [mL]            

In [46]:
data.head(10)

,Time [ms],Rel.Time [s],MVe [L/min],MVi [L/min],Cdyn [L/bar],R [mbar/L/s],MVespon [L/min],Rpat [mbar/L/s],MVemand [L/min],FlowDev [L/min],...,TC [s],TCe [s],C20/Cdyn [no unit],VTe [mL],VTi [mL],EIP [mbar],MVleak [L/min],Tispon [s],I:Espon (I-Part) [no unit],I:Espon (E-Part) [no unit]
Date_Time,,,,,,,,,,,,,,,,,,,,,
2020-11-02 13:42:39.904,1604324559904,0,0.2,0.21,0.21,146.0,0.0,128.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:40.029,1604324560029,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.11,0.55,3.4,3.4,21.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:40.951,1604324560951,1,0.2,0.21,0.22,146.0,0.0,128.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:41.060,1604324561060,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.21,0.55,8.2,3.6,22.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:41.935,1604324561935,2,0.2,0.21,0.26,154.0,0.0,136.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:42.060,1604324562060,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04,0.12,0.66,4.1,3.5,19.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:42.888,1604324562888,3,0.2,0.21,0.29,154.0,0.0,136.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:43.028,1604324563028,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.05,0.19,0.66,5.5,3.8,18.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:43.919,1604324563919,4,0.2,0.20,0.40,193.0,0.0,177.0,0.2,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# This is called a `hack`
# During `mean() na values are excluded by default

data = data.resample('1S').mean()
data.head(10)

,Time [ms],Rel.Time [s],MVe [L/min],MVi [L/min],Cdyn [L/bar],R [mbar/L/s],MVespon [L/min],Rpat [mbar/L/s],MVemand [L/min],FlowDev [L/min],...,TC [s],TCe [s],C20/Cdyn [no unit],VTe [mL],VTi [mL],EIP [mbar],MVleak [L/min],Tispon [s],I:Espon (I-Part) [no unit],I:Espon (E-Part) [no unit]
Date_Time,,,,,,,,,,,,,,,,,,,,,
2020-11-02 13:42:39,1.604325e+12,0.0,0.20,0.21,0.21,146.0,0.0,128.0,0.20,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:40,1.604325e+12,0.5,0.20,0.21,0.22,146.0,0.0,128.0,0.20,6.9,...,0.03,0.11,0.55,3.4,3.4,21.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:41,1.604325e+12,1.5,0.20,0.21,0.26,154.0,0.0,136.0,0.20,6.9,...,0.03,0.21,0.55,8.2,3.6,22.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:42,1.604325e+12,2.5,0.20,0.21,0.29,154.0,0.0,136.0,0.20,6.9,...,0.04,0.12,0.66,4.1,3.5,19.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:43,1.604325e+12,3.5,0.20,0.20,0.40,193.0,0.0,177.0,0.20,6.9,...,0.05,0.19,0.66,5.5,3.8,18.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:44,1.604325e+12,4.5,0.21,0.20,0.39,193.0,0.0,177.0,0.21,6.9,...,0.06,0.10,0.69,2.6,4.3,16.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:45,1.604325e+12,5.5,0.21,0.20,0.42,193.0,0.0,177.0,0.21,6.8,...,0.08,0.20,0.69,5.2,4.0,18.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:46,1.604325e+12,6.5,0.22,0.20,0.30,195.0,0.0,177.0,0.22,6.8,...,0.08,0.12,0.69,2.2,4.3,17.0,0.0,NaN,NaN,NaN
2020-11-02 13:42:47,1.604325e+12,7.5,0.22,0.21,0.34,195.0,0.0,177.0,0.22,6.8,...,0.06,0.12,0.59,3.5,3.6,19.0,0.0,NaN,NaN,NaN


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 344677 entries, 2020-11-02 13:42:39 to 2020-11-06 13:27:15
Freq: S
Data columns (total 45 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Time [ms]                   344677 non-null  float64
 1   Rel.Time [s]                344677 non-null  float64
 2   MVe [L/min]                 344519 non-null  float64
 3   MVi [L/min]                 344519 non-null  float64
 4   Cdyn [L/bar]                343959 non-null  float64
 5   R [mbar/L/s]                341986 non-null  float64
 6   MVespon [L/min]             344519 non-null  float64
 7   Rpat [mbar/L/s]             341285 non-null  float64
 8   MVemand [L/min]             344519 non-null  float64
 9   FlowDev [L/min]             344563 non-null  float64
 10  VTmand [mL]                 344453 non-null  float64
 11  r2 [no unit]                344424 non-null  float64
 12  VTispon [mL]                34

##### Please recognise that these are already aggregate data !!! The numbers in the same colum do not necessary belong to the same observation (e.g. ventilator inflations)

### 5. Remove na values

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 344677 entries, 2020-11-02 13:42:39 to 2020-11-06 13:27:15
Freq: S
Data columns (total 45 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Time [ms]                   344677 non-null  float64
 1   Rel.Time [s]                344677 non-null  float64
 2   MVe [L/min]                 344519 non-null  float64
 3   MVi [L/min]                 344519 non-null  float64
 4   Cdyn [L/bar]                343959 non-null  float64
 5   R [mbar/L/s]                341986 non-null  float64
 6   MVespon [L/min]             344519 non-null  float64
 7   Rpat [mbar/L/s]             341285 non-null  float64
 8   MVemand [L/min]             344519 non-null  float64
 9   FlowDev [L/min]             344563 non-null  float64
 10  VTmand [mL]                 344453 non-null  float64
 11  r2 [no unit]                344424 non-null  float64
 12  VTispon [mL]                34

In [50]:
data.shape

(344677, 45)

In [51]:
# Some columns are almost completely empty and hopeless - drop them

columns_to_drop = ['Tispon [s]', 'I:Espon (I-Part) [no unit]', 'I:Espon (E-Part) [no unit]']

data = data.drop(columns_to_drop, axis = 1)

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 344677 entries, 2020-11-02 13:42:39 to 2020-11-06 13:27:15
Freq: S
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Time [ms]           344677 non-null  float64
 1   Rel.Time [s]        344677 non-null  float64
 2   MVe [L/min]         344519 non-null  float64
 3   MVi [L/min]         344519 non-null  float64
 4   Cdyn [L/bar]        343959 non-null  float64
 5   R [mbar/L/s]        341986 non-null  float64
 6   MVespon [L/min]     344519 non-null  float64
 7   Rpat [mbar/L/s]     341285 non-null  float64
 8   MVemand [L/min]     344519 non-null  float64
 9   FlowDev [L/min]     344563 non-null  float64
 10  VTmand [mL]         344453 non-null  float64
 11  r2 [no unit]        344424 non-null  float64
 12  VTispon [mL]        344520 non-null  float64
 13  Pmin [mbar]         344563 non-null  float64
 14  Pmean [mbar]        344563 non-null  float

In [53]:
data.isnull().sum()

Time [ms]                 0
Rel.Time [s]              0
MVe [L/min]             158
MVi [L/min]             158
Cdyn [L/bar]            718
R [mbar/L/s]           2691
MVespon [L/min]         158
Rpat [mbar/L/s]        3392
MVemand [L/min]         158
FlowDev [L/min]         114
VTmand [mL]             224
r2 [no unit]            253
VTispon [mL]            157
Pmin [mbar]             114
Pmean [mbar]            114
PEEP [mbar]             114
RRmand [1/min]          114
PIP [mbar]              114
VTmand [L]              224
VTspon [L]              157
VTemand [mL]            224
VTespon [mL]            157
VTimand [mL]            220
VT [mL]                 224
% leak [%]              158
RRspon [1/min]        88501
% MVspon [%]            158
MV [L/min]              158
RRtrig [1/min]          158
RR [1/min]              158
I (I:E) [no unit]       119
E (I:E) [no unit]       119
FiO2 [%]                114
VTspon [mL]            1359
E [mbar/L]             5704
TC [s]              

In [54]:
# How many data points are missing ?

data.isnull().sum()

Time [ms]                 0
Rel.Time [s]              0
MVe [L/min]             158
MVi [L/min]             158
Cdyn [L/bar]            718
R [mbar/L/s]           2691
MVespon [L/min]         158
Rpat [mbar/L/s]        3392
MVemand [L/min]         158
FlowDev [L/min]         114
VTmand [mL]             224
r2 [no unit]            253
VTispon [mL]            157
Pmin [mbar]             114
Pmean [mbar]            114
PEEP [mbar]             114
RRmand [1/min]          114
PIP [mbar]              114
VTmand [L]              224
VTspon [L]              157
VTemand [mL]            224
VTespon [mL]            157
VTimand [mL]            220
VT [mL]                 224
% leak [%]              158
RRspon [1/min]        88501
% MVspon [%]            158
MV [L/min]              158
RRtrig [1/min]          158
RR [1/min]              158
I (I:E) [no unit]       119
E (I:E) [no unit]       119
FiO2 [%]                114
VTspon [mL]            1359
E [mbar/L]             5704
TC [s]              

In [55]:
# How many percent of data is missing

data.isnull().sum() / len(data) * 100

Time [ms]              0.000000
Rel.Time [s]           0.000000
MVe [L/min]            0.045840
MVi [L/min]            0.045840
Cdyn [L/bar]           0.208311
R [mbar/L/s]           0.780731
MVespon [L/min]        0.045840
Rpat [mbar/L/s]        0.984110
MVemand [L/min]        0.045840
FlowDev [L/min]        0.033074
VTmand [mL]            0.064988
r2 [no unit]           0.073402
VTispon [mL]           0.045550
Pmin [mbar]            0.033074
Pmean [mbar]           0.033074
PEEP [mbar]            0.033074
RRmand [1/min]         0.033074
PIP [mbar]             0.033074
VTmand [L]             0.064988
VTspon [L]             0.045550
VTemand [mL]           0.064988
VTespon [mL]           0.045550
VTimand [mL]           0.063828
VT [mL]                0.064988
% leak [%]             0.045840
RRspon [1/min]        25.676503
% MVspon [%]           0.045840
MV [L/min]             0.045840
RRtrig [1/min]         0.045840
RR [1/min]             0.045840
I (I:E) [no unit]      0.034525
E (I:E) 

#### A lot of things are happening here, for example vectorized computation, broadcasting We will speak about them during the next session.

### 6. Now let us save the processed data

#### A. export them as `csv` files

In [57]:
%%time

path = os.path.join('results', 'data_processed')
data.to_csv(path)

CPU times: user 10.5 s, sys: 228 ms, total: 10.7 s
Wall time: 10.8 s


#### B. export az Excel file

This will run for a long time

#### C. export them as serialised binary data  - `pickle`

In [58]:
%%time

import pickle

path = os.path.join('results', 'data_processed.pickle')
filehandle = open(path, 'wb')
pickle.dump(data, filehandle)
filehandle.close()

CPU times: user 34.4 ms, sys: 88.5 ms, total: 123 ms
Wall time: 198 ms


In [59]:
%%time

import pickle

path = os.path.join('results', 'data_processed.pickle')
filehandle = open(os.path.join(path), 'rb')
data_processed = pickle.load(filehandle)
filehandle.close()

data_processed

CPU times: user 29.6 ms, sys: 62.9 ms, total: 92.5 ms
Wall time: 93.3 ms


,Time [ms],Rel.Time [s],MVe [L/min],MVi [L/min],Cdyn [L/bar],R [mbar/L/s],MVespon [L/min],Rpat [mbar/L/s],MVemand [L/min],FlowDev [L/min],...,FiO2 [%],VTspon [mL],E [mbar/L],TC [s],TCe [s],C20/Cdyn [no unit],VTe [mL],VTi [mL],EIP [mbar],MVleak [L/min]
Date_Time,,,,,,,,,,,,,,,,,,,,,
2020-11-02 13:42:39,1.604325e+12,0.0,0.20,0.21,0.21,146.0,0.0,128.0,0.20,6.9,...,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-02 13:42:40,1.604325e+12,0.5,0.20,0.21,0.22,146.0,0.0,128.0,0.20,6.9,...,25.0,0.0,4774.0,0.03,0.11,0.55,3.4,3.4,21.0,0.00
2020-11-02 13:42:41,1.604325e+12,1.5,0.20,0.21,0.26,154.0,0.0,136.0,0.20,6.9,...,25.0,0.0,4597.0,0.03,0.21,0.55,8.2,3.6,22.0,0.00
2020-11-02 13:42:42,1.604325e+12,2.5,0.20,0.21,0.29,154.0,0.0,136.0,0.20,6.9,...,25.0,0.0,3798.0,0.04,0.12,0.66,4.1,3.5,19.0,0.00
2020-11-02 13:42:43,1.604325e+12,3.5,0.20,0.20,0.40,193.0,0.0,177.0,0.20,6.9,...,25.0,0.0,3408.0,0.05,0.19,0.66,5.5,3.8,18.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-06 13:27:11,1.604669e+12,344671.5,0.22,0.21,0.22,211.0,0.0,193.0,0.22,6.8,...,26.0,0.0,4502.0,0.07,0.12,0.61,3.5,3.7,22.0,0.00
2020-11-06 13:27:12,1.604669e+12,344672.5,0.23,0.22,0.26,164.0,0.0,150.0,0.22,6.9,...,26.0,0.0,3879.0,0.05,0.16,0.61,4.7,3.7,22.0,0.01
2020-11-06 13:27:13,1.604669e+12,344673.5,0.22,0.22,0.21,164.0,0.0,150.0,0.22,6.9,...,26.0,0.0,4695.0,0.05,0.21,0.75,6.3,4.0,21.0,0.02


##### To be continued... 

### 6. Homework

### 1. Further explorative analysis on data

In [ ]:
# Select all data during the 1 minute period at 2020-11-03 13:00 

selection = data
selection

In [ ]:
# Select all data between 2020-11-03 13:00 and and 15:00

selection = data
selection

In [ ]:
# Select all data between 2020-11-03 13:00 and and 15:00 and limit it to 
# "MVe [L/min]" and "MVi [L/min]" columns only

selection = data
selection

### 2. Import and process the `Radiometer_TCM_sample_data.xlsx'` file containing transcuatenous CO2 data into a Dataframe named "data_2"

The file is located in the "data" subfolder

*Hint: the first 8 rows contain text rather than data and need to be skipped*

Check the data types of the various columns

Now the 'Time` column seems to be in the correct format. Set it as index

 ...but most other columns are just objects. First check what values do the columns contain

... to be continued